# Title
[]()

In [15]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

# Update scripts

In [16]:
references_df_dict = dict()

## iteration 1

In [5]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import text
from sqlalchemy import Column, ForeignKey, Integer, String, Text, TIMESTAMP, Numeric, Boolean
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

## This script calls upon functions and magic functions in db_session.py

Base = declarative_base()

class GPT_queue(Base):
    __tablename__ = 'gpt_queue'
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    body = mapped_column(Text)
    section = mapped_column(String(100))
    sent_to_sources = mapped_column(Boolean)
    publication = mapped_column(String(100))

class Sources(Base):
    __tablename__ = 'sources'
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    text = mapped_column(Text)
    abstract = mapped_column(Text)
    publication = mapped_column(String(100))
    authors = mapped_column(String(300))
    year = mapped_column(Integer)
    month = mapped_column(String(10))
    pub_volume = mapped_column(String(10))
    pub_issue = mapped_column(String(10))
    start_page = mapped_column(String(10))
    end_page = mapped_column(String(10))
    doi = mapped_column(String(50))
    section = mapped_column(String(100))
    summaries = relationship('Summaries', back_populates='sources')

class Prompts(Base):
    __tablename__ = 'prompts'
    id = mapped_column(Integer, primary_key=True)
    full_template = mapped_column(Text)
    system_role = mapped_column(String(300))
    prep_steps = mapped_column(Text)
    task = mapped_column(Text)
    edit_steps = mapped_column(Text)
    simplify_steps = mapped_column(Text)
    audience = mapped_column(String(200))
    format_steps = mapped_column(Text)

    summaries = relationship('Summaries', back_populates='prompts')
    
class Summaries(Base):
    __tablename__ = 'summaries'
    id = mapped_column(Integer, primary_key=True)
    timestamp = mapped_column(TIMESTAMP(timezone=True))
    original_summary = mapped_column(Text)
    rating_original_content = mapped_column(Integer) 
    simple_summary = mapped_column(Text)
    rating_simple_content = mapped_column(Integer) 
    original_headline = mapped_column(String(255))
    prompt_id = mapped_column(Integer, ForeignKey('prompts.id'), autoincrement=False)
    reference_id = mapped_column(Integer, ForeignKey('sources.id'), autoincrement=False)
    choice = mapped_column(Integer)
    model = mapped_column(String(70))
    temperature = mapped_column(Numeric)

    prompts = relationship('Prompts', back_populates='summaries')
    sources = relationship('Sources', back_populates='summaries')

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df

@remote_sql_session
def get_table(session, query='SELECT *', table='publications', limit=None, order_by='id', order='ASC'):
    """
    Return a database table as a pandas dataframe.
    """
    query_statement = f'{query} from {table}'
    if order_by:
        query_statement += f' ORDER BY {order_by} {order}'
    if limit:
        query_statement += f' LIMIT {limit}'
    print(f'Query: {query_statement}')
    q = session.execute(text(query_statement))
    df = pd.DataFrame(q.fetchall())
    return df


def bulk_append(input_df, table='summaries'):
    """
    Add articles to the `sources` table in the database from a dataframe containing article text and metadata.
    
    Parameters:
    - references_df: pandas dataframe containing article text and metadata.

    Returns: None
    """
    @remote_sql_session
    def insert_rows(session):
        try:
            print(f'Adding {len(input_df)} rows to the database...')
            def insert_row(row):
                if table == 'sources':
                    with session.no_autoflush:
                        existing_record = session.query(Sources).filter_by(
                            title=row['title'],
                            doi=row['doi'],
                            section=row['section']
                        ).first()
                        if not existing_record:
                            data = Sources(
                                title=row['title'],
                                text=row['text'],
                                abstract=row['abstract'],
                                publication=row['publication'],
                                authors=row['authors'],
                                year=row['year'],
                                month=row['month'],
                                pub_volume=row['pub_volume'],
                                pub_issue=row['pub_issue'],
                                start_page=row['start_page'],
                                end_page=row['end_page'],
                                doi=row['doi'],
                                section=row['section'] 
                            )
                            session.add(data)
                            print(f'\t{row["title"]}')
                        else:
                            print(f'\t** Already exists in the database: {row["title"]}.')
                elif table == 'gpt_queue':
                    data = GPT_queue(
                        title=row['title'],
                        body=row['body'],
                        section=row['section'],
                        sent_to_sources=row['sent_to_sources'],
                        publication=row['publication']
                    )
                    session.add(data)
                    print(f'\t{row["title"]}')
                elif table == 'summaries':
                    prompt = session.query(Prompts).filter_by(
                        full_template=row['full_summarize_task'],
                        system_role=row['system_role'],
                    ).first()
                    if prompt:
                        prompt_id = prompt.id
                    else:
                        prompt = Prompts(
                            full_template=row['full_summarize_task'],
                            prep_steps=row['prep_step'],
                            task=row['summarize_task'],
                            edit_steps=row['edit_task'],
                            audience=row['simplify_audience'],
                            simplify_steps=row['simplify_task'],
                            format_steps=row['format_task'],
                            system_role=row['system_role']
                        )
                        session.add(prompt)
                        session.flush()
                        prompt_id = prompt.id

                    summary = Summaries(
                        timestamp=row['timestamp'],
                        original_summary=row['summary'],
                        simple_summary=row['simple_summary'],
                        original_headline=row['headline'],
                        prompt_id=prompt_id,
                        reference_id=row['reference_id'],
                        choice=row['choice'],
                        model=row['model'],
                        temperature=row['temperature']
                    )
                    session.add(summary)
                    print(f'\tReference #{row["reference_id"]}: {row["headline"]}')
                elif table == 'feed':
                    source = session.query(Feed).filter_by(
                        title=row['title'],
                        journal=row['journal'],
                        doi=row['doi']
                    ).first()
                    if source:
                        print(f'\tAlready exists in the database: {row["title"]}.')

            input_df.apply(insert_row, axis=1)

            session.commit()
            print("New records added successfully (if applicable)!")
        except Exception as e:
            session.rollback()
            print(f"Error adding data to the database: {str(e)}")
        finally:
            session.close()

    return insert_rows()

from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
#########
# Prep: Set parameters
folder_path = '../text/2023-05-03 5'
section = 'Sleep disturbance and obesity risk'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
    # return references_df_dict[iteration_id]

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=local, article_limit=article_limit)


Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])
Number of abstract sections: 3
Adding 1 rows to the database...
	Sleep and obesity
Error adding data to the database: (psycopg2.ProgrammingError) can't adapt type 'Series'
[SQL: INSERT INTO sources (title, text, abstract, publication, authors, year, month, pub_volume, pub_issue, start_page, end_page, doi, section) VALUES (%(title)s, %(text)s, %(abstract)s, %(publication)s, %(authors)s, %(year)s, %(month)s, %(pub_volume)s, %(pub_issue)s, %(start_page)s, %(end_page)s, %(doi)s, %(section)s) RETURNING sources.id]
[parameters: {'title': 'Sleep and obesity', 'text': 'Sleep and Obesity\r\n\r\nObstructive sleep apnea\r\n\r\nSleep loss occurs not only as a result of habitual behavior, but also in presence of patholog ... (4122 characters truncated) ...  self-reported and the analysis could not control the presence of a sleep disorder like OSA, which could in pa

IndexError: list index out of range

In [ ]:
qna_dict[iteration_id]

,id,title,body,sent_to_sources,section,publication,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text
0,87,Sleep and obesity,Sleep and Obesity\r\n\r\nObstructive sleep apn...,None,Sleep disturbance and obesity risk,Curr Opin Clin Nutr Metab Care,PURPOSE OF REVIEW: This review summarizes the ...,Current opinion in clinical nutrition and meta...,"Guglielmo Beccuti, Silvana Pannain",2011,,14,4,402,412,10.1097/MCO.0b013e3283479109,Sleep and Obesity\r\n\r\nObstructive sleep apn...


In [8]:
for column in qna_dict.columns:
    print(qna_dict[column].values)

[87]
['Sleep and obesity']
['Sleep and Obesity\r\n\r\nObstructive sleep apnea\r\n\r\nSleep loss occurs not only as a result of habitual behavior, but also in presence of pathological conditions associated with disturbed sleep, like obstructive sleep apnea (OSA). The increase in both the prevalence and the severity of obesity has translated into an increase in the prevalence of obesity-related comorbidities including OSA. The prevalence of OSA in the US adult population has been estimated to be 24% in men and 9% in women [49] but is increased in severe obesity by up to 93.6% among men and 73.5% among women [50].\r\n\r\nPoor sleep quality and excessive daytime sleepiness in the absence of obstructive sleep apnea\r\n\r\nSevere obesity appears to be associated with marked sleep disturbances, even in individuals who do not have OSA [72–75]. Such sleep disturbances may equally predispose severely obese individuals to accumulate a sleep debt and may contribute to the dysregulation of appetite

## Iteration 2

In [20]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import text
from sqlalchemy import Column, ForeignKey, Integer, String, Text, TIMESTAMP, Numeric, Boolean
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

## This script calls upon functions and magic functions in db_session.py

Base = declarative_base()

class GPT_queue(Base):
    __tablename__ = 'gpt_queue'
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    body = mapped_column(Text)
    section = mapped_column(String(100))
    sent_to_sources = mapped_column(Boolean)
    publication = mapped_column(String(100))

class Sources(Base):
    __tablename__ = 'sources'
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    text = mapped_column(Text)
    abstract = mapped_column(Text)
    publication = mapped_column(String(100))
    authors = mapped_column(String(300))
    year = mapped_column(Integer)
    month = mapped_column(String(10))
    pub_volume = mapped_column(String(10))
    pub_issue = mapped_column(String(10))
    start_page = mapped_column(String(10))
    end_page = mapped_column(String(10))
    doi = mapped_column(String(50))
    section = mapped_column(String(100))
    summaries = relationship('Summaries', back_populates='sources')

class Prompts(Base):
    __tablename__ = 'prompts'
    id = mapped_column(Integer, primary_key=True)
    full_template = mapped_column(Text)
    system_role = mapped_column(String(300))
    prep_steps = mapped_column(Text)
    task = mapped_column(Text)
    edit_steps = mapped_column(Text)
    simplify_steps = mapped_column(Text)
    audience = mapped_column(String(200))
    format_steps = mapped_column(Text)

    summaries = relationship('Summaries', back_populates='prompts')
    
class Summaries(Base):
    __tablename__ = 'summaries'
    id = mapped_column(Integer, primary_key=True)
    timestamp = mapped_column(TIMESTAMP(timezone=True))
    original_summary = mapped_column(Text)
    rating_original_content = mapped_column(Integer) 
    simple_summary = mapped_column(Text)
    rating_simple_content = mapped_column(Integer) 
    original_headline = mapped_column(String(255))
    prompt_id = mapped_column(Integer, ForeignKey('prompts.id'), autoincrement=False)
    reference_id = mapped_column(Integer, ForeignKey('sources.id'), autoincrement=False)
    choice = mapped_column(Integer)
    model = mapped_column(String(70))
    temperature = mapped_column(Numeric)

    prompts = relationship('Prompts', back_populates='summaries')
    sources = relationship('Sources', back_populates='summaries')

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df

@remote_sql_session
def get_table(session, query='SELECT *', table='publications', limit=None, order_by='id', order='ASC'):
    """
    Return a database table as a pandas dataframe.
    """
    query_statement = f'{query} from {table}'
    if order_by:
        query_statement += f' ORDER BY {order_by} {order}'
    if limit:
        query_statement += f' LIMIT {limit}'
    print(f'Query: {query_statement}')
    q = session.execute(text(query_statement))
    df = pd.DataFrame(q.fetchall())
    return df


def bulk_append(input_df, table='summaries'):
    """
    Add articles to the `sources` table in the database from a dataframe containing article text and metadata.
    
    Parameters:
    - references_df: pandas dataframe containing article text and metadata.

    Returns: None
    """
    @remote_sql_session
    def insert_rows(session):
        try:
            print(f'Adding {len(input_df)} rows to the database...')
            def insert_row(row):
                if table == 'sources':
                    with session.no_autoflush:
                        existing_record = session.query(Sources).filter_by(
                            title=row['title'],
                            doi=row['doi'],
                            section=row['section']
                        ).first()
                        if not existing_record:
                            data = Sources(
                                title=row['title'],
                                text=row['text'],
                                abstract=row['abstract'],
                                publication=row['journal'],
                                authors=row['authors'],
                                year=row['year'],
                                month=row['month'],
                                pub_volume=row['pub_volume'],
                                pub_issue=row['pub_issue'],
                                start_page=row['start_page'],
                                end_page=row['end_page'],
                                doi=row['doi'],
                                section=row['section'] 
                            )
                            session.add(data)
                            print(f'\t{row["title"]}')
                        else:
                            print(f'\t** Already exists in the database: {row["title"]}.')
                elif table == 'gpt_queue':
                    data = GPT_queue(
                        title=row['title'],
                        body=row['body'],
                        section=row['section'],
                        sent_to_sources=row['sent_to_sources'],
                        publication=row['journal']
                    )
                    session.add(data)
                    print(f'\t{row["title"]}')
                elif table == 'summaries':
                    prompt = session.query(Prompts).filter_by(
                        full_template=row['full_summarize_task'],
                        system_role=row['system_role'],
                    ).first()
                    if prompt:
                        prompt_id = prompt.id
                    else:
                        prompt = Prompts(
                            full_template=row['full_summarize_task'],
                            prep_steps=row['prep_step'],
                            task=row['summarize_task'],
                            edit_steps=row['edit_task'],
                            audience=row['simplify_audience'],
                            simplify_steps=row['simplify_task'],
                            format_steps=row['format_task'],
                            system_role=row['system_role']
                        )
                        session.add(prompt)
                        session.flush()
                        prompt_id = prompt.id

                    summary = Summaries(
                        timestamp=row['timestamp'],
                        original_summary=row['summary'],
                        simple_summary=row['simple_summary'],
                        original_headline=row['headline'],
                        prompt_id=prompt_id,
                        reference_id=row['reference_id'],
                        choice=row['choice'],
                        model=row['model'],
                        temperature=row['temperature']
                    )
                    session.add(summary)
                    print(f'\tReference #{row["reference_id"]}: {row["headline"]}')
                elif table == 'feed':
                    source = session.query(Feed).filter_by(
                        title=row['title'],
                        journal=row['journal'],
                        doi=row['doi']
                    ).first()
                    if source:
                        print(f'\tAlready exists in the database: {row["title"]}.')

            input_df.apply(insert_row, axis=1)

            session.commit()
            print("New records added successfully (if applicable)!")
        except Exception as e:
            session.rollback()
            print(f"Error adding data to the database: {str(e)}")
        finally:
            session.close()

    return insert_rows()

from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
#########
# Prep: Set parameters
folder_path = '../text/2023-05-03 5'
section = 'Sleep disturbance and obesity risk'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
    # return references_df_dict[iteration_id]

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]


import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
import re
import os
import string
import pandas as pd
import requests
from article_processing import create_text_dict_from_folder
from orm_summarize import *
api_key = os.getenv('api_ncbi') # Pubmed API key

### These scripts populate data in the sources table with data from the Pubmed API.

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)
    # print(f'Data keys: {data.keys()}')
    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
    print(f'Number of abstract sections: {len(abstract_matches)}')
    if len(abstract_matches) > 1:
        cleaned_abstract_sections = []
        for match in abstract_matches:
            # clean_match = re.sub(r'<AbstractText(.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
            cleaned_abstract_sections.append(clean_match)
            
        abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
    else:
        abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''

    # abstract = 'abstract'

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'journal': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'journal': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df


iteration_id = 2
text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py

references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
references_df_dict[iteration_id]['abstract'].values

Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])
Number of abstract sections: 3


array(['PURPOSE OF REVIEW: This review summarizes the most recent evidence linking decreased sleep duration and poor sleep quality to obesity, focusing upon studies in adults.<br>RECENT FINDINGS: Published and unpublished health examination surveys and epidemiological studies suggest that the worldwide prevalence of obesity has doubled since 1980. In 2008, 1 in 10 adults was obese, with women more likely to be obese than men. This obesity epidemic has been paralleled by a trend of reduced sleep duration. Poor sleep quality, which leads to overall sleep loss has also become a frequent complaint. Growing evidence from both laboratory and epidemiological studies points to short sleep duration and poor sleep quality as new risk factors for the development of obesity.<br>SUMMARY: Sleep is an important modulator of neuroendocrine function and glucose metabolism and sleep loss has been shown to result in metabolic and endocrine alterations, including decreased glucose tolerance, decreased ins

In [13]:
references_df_dict[iteration_id]

,id,title,body,sent_to_sources,section,publication,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text
0,87,Sleep and obesity,Sleep and Obesity\r\n\r\nObstructive sleep apn...,None,Sleep disturbance and obesity risk,Curr Opin Clin Nutr Metab Care,PURPOSE OF REVIEW: This review summarizes the ...,Current opinion in clinical nutrition and meta...,"Guglielmo Beccuti, Silvana Pannain",2011,,14,4,402,412,10.1097/MCO.0b013e3283479109,Sleep and Obesity\r\n\r\nObstructive sleep apn...


## 2.1

In [15]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
import re
import os
import string
import pandas as pd
import requests
from article_processing import create_text_dict_from_folder
from orm_summarize import *
api_key = os.getenv('api_ncbi') # Pubmed API key

### These scripts populate data in the sources table with data from the Pubmed API.

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)
    # print(f'Data keys: {data.keys()}')
    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
    print(f'Number of abstract sections: {len(abstract_matches)}')
    if len(abstract_matches) > 1:
        cleaned_abstract_sections = []
        for match in abstract_matches:
            # clean_match = re.sub(r'<AbstractText(.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
            cleaned_abstract_sections.append(clean_match)
            
        abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
    else:
        abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''

    # abstract = 'abstract'

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'journal': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'journal': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df


import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import text
from sqlalchemy import Column, ForeignKey, Integer, String, Text, TIMESTAMP, Numeric, Boolean
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

## This script calls upon functions and magic functions in db_session.py

Base = declarative_base()

class GPT_queue(Base):
    __tablename__ = 'gpt_queue'
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    body = mapped_column(Text)
    section = mapped_column(String(100))
    sent_to_sources = mapped_column(Boolean)
    publication = mapped_column(String(100))

class Sources(Base):
    __tablename__ = 'sources'
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    text = mapped_column(Text)
    abstract = mapped_column(Text)
    publication = mapped_column(String(100))
    authors = mapped_column(String(300))
    year = mapped_column(Integer)
    month = mapped_column(String(10))
    pub_volume = mapped_column(String(10))
    pub_issue = mapped_column(String(10))
    start_page = mapped_column(String(10))
    end_page = mapped_column(String(10))
    doi = mapped_column(String(50))
    section = mapped_column(String(100))
    summaries = relationship('Summaries', back_populates='sources')

class Prompts(Base):
    __tablename__ = 'prompts'
    id = mapped_column(Integer, primary_key=True)
    full_template = mapped_column(Text)
    system_role = mapped_column(String(300))
    prep_steps = mapped_column(Text)
    task = mapped_column(Text)
    edit_steps = mapped_column(Text)
    simplify_steps = mapped_column(Text)
    audience = mapped_column(String(200))
    format_steps = mapped_column(Text)

    summaries = relationship('Summaries', back_populates='prompts')
    
class Summaries(Base):
    __tablename__ = 'summaries'
    id = mapped_column(Integer, primary_key=True)
    timestamp = mapped_column(TIMESTAMP(timezone=True))
    original_summary = mapped_column(Text)
    rating_original_content = mapped_column(Integer) 
    simple_summary = mapped_column(Text)
    rating_simple_content = mapped_column(Integer) 
    original_headline = mapped_column(String(255))
    prompt_id = mapped_column(Integer, ForeignKey('prompts.id'), autoincrement=False)
    reference_id = mapped_column(Integer, ForeignKey('sources.id'), autoincrement=False)
    choice = mapped_column(Integer)
    model = mapped_column(String(70))
    temperature = mapped_column(Numeric)

    prompts = relationship('Prompts', back_populates='summaries')
    sources = relationship('Sources', back_populates='summaries')

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df

@remote_sql_session
def get_table(session, query='SELECT *', table='publications', limit=None, order_by='id', order='ASC'):
    """
    Return a database table as a pandas dataframe.
    """
    query_statement = f'{query} from {table}'
    if order_by:
        query_statement += f' ORDER BY {order_by} {order}'
    if limit:
        query_statement += f' LIMIT {limit}'
    print(f'Query: {query_statement}')
    q = session.execute(text(query_statement))
    df = pd.DataFrame(q.fetchall())
    return df


def bulk_append(input_df, table='summaries'):
    """
    Add articles to the `sources` table in the database from a dataframe containing article text and metadata.
    
    Parameters:
    - references_df: pandas dataframe containing article text and metadata.

    Returns: None
    """
    @remote_sql_session
    def insert_rows(session):
        try:
            print(f'Adding {len(input_df)} rows to the database...')
            def insert_row(row):
                if table == 'sources':
                    with session.no_autoflush:
                        existing_record = session.query(Sources).filter_by(
                            title=row['title'],
                            doi=row['doi'],
                            section=row['section']
                        ).first()
                        if not existing_record:
                            data = Sources(
                                title=row['title'],
                                text=row['text'],
                                abstract=row['abstract'],
                                publication=row['journal'],
                                authors=row['authors'],
                                year=row['year'],
                                month=row['month'],
                                pub_volume=row['pub_volume'],
                                pub_issue=row['pub_issue'],
                                start_page=row['start_page'],
                                end_page=row['end_page'],
                                doi=row['doi'],
                                section=row['section'] 
                            )
                            session.add(data)
                            print(f'\t{row["title"]}')
                        else:
                            print(f'\t** Already exists in the database: {row["title"]}.')
                elif table == 'gpt_queue':
                    data = GPT_queue(
                        title=row['title'],
                        body=row['body'],
                        section=row['section'],
                        sent_to_sources=row['sent_to_sources'],
                        publication=row['journal']
                    )
                    session.add(data)
                    print(f'\t{row["title"]}')
                elif table == 'summaries':
                    prompt = session.query(Prompts).filter_by(
                        full_template=row['full_summarize_task'],
                        system_role=row['system_role'],
                    ).first()
                    if prompt:
                        prompt_id = prompt.id
                    else:
                        prompt = Prompts(
                            full_template=row['full_summarize_task'],
                            prep_steps=row['prep_step'],
                            task=row['summarize_task'],
                            edit_steps=row['edit_task'],
                            audience=row['simplify_audience'],
                            simplify_steps=row['simplify_task'],
                            format_steps=row['format_task'],
                            system_role=row['system_role']
                        )
                        session.add(prompt)
                        session.flush()
                        prompt_id = prompt.id

                    summary = Summaries(
                        timestamp=row['timestamp'],
                        original_summary=row['summary'],
                        simple_summary=row['simple_summary'],
                        original_headline=row['headline'],
                        prompt_id=prompt_id,
                        reference_id=row['reference_id'],
                        choice=row['choice'],
                        model=row['model'],
                        temperature=row['temperature']
                    )
                    session.add(summary)
                    print(f'\tReference #{row["reference_id"]}: {row["headline"]}')
                elif table == 'feed':
                    source = session.query(Feed).filter_by(
                        title=row['title'],
                        journal=row['journal'],
                        doi=row['doi']
                    ).first()
                    if source:
                        print(f'\tAlready exists in the database: {row["title"]}.')

            input_df.apply(insert_row, axis=1)

            session.commit()
            print("New records added successfully (if applicable)!")
        except Exception as e:
            session.rollback()
            print(f"Error adding data to the database: {str(e)}")
        finally:
            session.close()

    return insert_rows()

from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
#########
# Prep: Set parameters
folder_path = '../text/2023-05-03 5'
section = 'Sleep disturbance and obesity risk'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 2.1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
    # return references_df_dict[iteration_id]

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]



if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=local, article_limit=article_limit)


Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])
Number of abstract sections: 3
Adding 1 rows to the database...
	** Already exists in the database: Sleep and obesity.
New records added successfully (if applicable)!
**Text #202 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 1 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
Processing 202_prompt00...
Original summaries DataFrame shape: (1, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       

In [16]:
qna_dict

,timestamp,reference_id,article_title,choice,text,system_role,model,temperature,prep_step,summarize_task,edit_task,simplify_task,simplify_audience,format_task,full_summarize_task,folder,summary,headline,simple_summary
0,2023-08-07 11:38:17.841662-07:00,202,Sleep and obesity,1,Sleep and Obesity\r\n\r\nObstructive sleep apn...,You are an expert at science communication.,gpt-3.5-turbo-16k-0613,1,Describe the interesting points to your cowork...,Take the key points and numerical descriptors to,,,,4. Return your final response in a JSON format...,Take the key points and numerical descriptors ...,text/2023-05-03 5,"Sleep loss and poor sleep quality, even in the...",None,None


## 2.2

In [1]:
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
import re
import os
import string
import pandas as pd
import requests
from article_processing import create_text_dict_from_folder
from orm_summarize import *
api_key = os.getenv('api_ncbi') # Pubmed API key

### These scripts populate data in the sources table with data from the Pubmed API.

def search_article(title, publication, api_key, verbose=False):
    """
    Search for article title in PubMed database.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    response (str): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi'
    title_without_not = re.sub(r'not', '', title)
    if api_key:
        base_url += f'&api_key={api_key}'
    if publication:
        search_term = f'({title_without_not} [ti]) AND ({publication} [ta])'
    else:
        search_term = f'{title_without_not} [ti]'
    params = {
        'db': 'pubmed',
        'term': search_term,
        'retmax': 5,
        'retmode': 'json'
    }
    print(f'Search term: {search_term}')

    response = requests.get(base_url, params=params)
    data = response.json()

    cleaned_title = re.sub(r'</?[ib]>', '', title) # remove bold and italic html tags
    cleaned_title = re.sub(r'[^a-zA-Z0-9 ]', '', cleaned_title).lower().strip()
    cleaned_title = re.sub(r"\u2010", '', cleaned_title)
    try:
        id_list = data['esearchresult']['idlist']
        if id_list:
            for index in range(len(id_list)):
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                cleaned_result = re.sub(r'[^a-zA-Z0-9 <>/]', '', result).lower().strip() 
                result = retrieve_citation(id_list[index], api_key).decode('utf-8')
                result_title_match = re.search(r'<articletitle>(.*?)</articletitle>', cleaned_result)
                if result_title_match:
                    result_title = result_title_match.group(1)
                    cleaned_result_title = re.sub(r'</?[ib]>', '', result_title)
                    cleaned_result_title = re.sub(r'/(?![^<>]*>)', '', cleaned_result_title) # Remove any / that is not within html tag
                    cleaned_result_title = re.sub(r'[^a-zA-Z0-9 <>/]', '', cleaned_result_title).lower().strip()
                else:
                    cleaned_result_title = cleaned_result
                if cleaned_title == cleaned_result_title:
                    if verbose:
                        print(f'Match found for {title}: PMID = {article_id}.')
                        return result
                else:
                    continue
            if cleaned_title != cleaned_result_title:
                print(f'Warning: Article title not found in PMIDs.')
                print(f'Check these PMIDs: {id_list}')
                print(f'\tInput title: {title.lower().strip()}')
                print(f'\tResult title: {result_title if result_title else cleaned_result}')
                print(f'\tCleaned input title: {cleaned_title}')
                print(f'\tCleaned result title: {cleaned_result_title}\n')
            return result     
    except Exception as error: 
        print(f'Response: \n{data}')
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('Article not found.')
        print(f'\tInput title: {title.lower().strip()}')
        return ''.join([id for id in id_list]) 
    
def retrieve_citation(article_id, api_key):
    """
    Retrieve article metadata from PubMed database.
    """
    base_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
    if api_key:
        base_url += f'&api_key={api_key}'
    params = {
        'db': 'pubmed',
        'id': article_id
    }

    response = requests.get(base_url, params=params)
    return response.content

def extract_pubmed_details(record_string):
    """
    Helper function called by `pubmed_details_by_title` to parse article metadata from PubMed database.
    """
    authors = re.findall(r'<Author ValidYN="Y".*?><LastName>(.*?)</LastName><ForeName>(.*?)</ForeName>', record_string)
    formatted_authors = ', '.join(['{} {}'.format(author[1], author[0]) for author in authors])

    # Extract publication year
    publication_year = re.search(r'<PubDate><Year>(\d{4})</Year>', record_string)
    publication_year = publication_year.group(1) if publication_year else ''
    publication_month = re.search(r'<PubDate>.*?<Month>(Aug)</Month>.*?</PubDate>', record_string)
    publication_month = publication_month.group(1) if publication_month else ''

    # Extract article title
    article_title = re.search(r'<ArticleTitle>(.*?)</ArticleTitle>', record_string)
    article_title = article_title.group(1) if article_title else ''

    # Extract journal title
    journal_title = re.search(r'<Title>(.*?)</Title>', record_string)
    journal_title = journal_title.group(1) if journal_title else ''

    # Extract journal volume
    journal_volume = re.search(r'<Volume>(.*?)</Volume>', record_string)
    journal_volume = journal_volume.group(1) if journal_volume else ''

    # Extract journal issue
    journal_issue = re.search(r'<Issue>(.*?)</Issue>', record_string)
    journal_issue = journal_issue.group(1) if journal_issue else ''

    # Extract start page
    start_page = re.search(r'<StartPage>(.*?)</StartPage>', record_string)
    start_page = start_page.group(1) if start_page else ''

    # Extract end page
    end_page = re.search(r'<EndPage>(.*?)</EndPage>', record_string)
    end_page = end_page.group(1) if end_page else ''

    # Extract ELocationID
    doi = re.search(r'<ELocationID.*?EIdType="doi".*?>(.*?)</ELocationID>', record_string)
    doi = doi.group(1) if doi else ''

    abstract_matches = re.findall(r'(<AbstractText.*?>.*?</AbstractText>)', record_string)
    print(f'Number of abstract sections: {len(abstract_matches)}')
    if len(abstract_matches) > 1:
        cleaned_abstract_sections = []
        for match in abstract_matches:
            # clean_match = re.sub(r'<AbstractText(.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'<AbstractText.*?((?:Label=".*")?.*?>.*)</AbstractText>', r'\1', match)
            clean_match = re.sub(r'(?: Label="(.*?)")?.*?>(.*)', r'\1: \2', clean_match)
            cleaned_abstract_sections.append(clean_match)
            
        abstract = ''.join([f'{group}<br>' for group in cleaned_abstract_sections])
    else:
        abstract = re.sub(r'<AbstractText.*?>(.*?)</AbstractText>', r'\1', abstract_matches[0])  if abstract_matches else ''

    return {
        'pubmed_title': article_title,
        'abstract': abstract,
        'journal': journal_title,
        'authors': formatted_authors,
        'year': publication_year,
        'month': publication_month,
        'pub_volume': journal_volume,
        'pub_issue': journal_issue,
        'start_page': start_page,
        'end_page': end_page,
        'doi': doi,
    }


def pubmed_details_by_title(title, publication, api_key):
    """
    Search for article title in PubMed database and return article details.

    Parameters:
    - title (str): article title
    - api_key (str): NCBI API key

    Returns:
    article_details (dict): Article metadata from PubMed database if present. Otherwise, returns list of PMIDs.
    """
    record_string = search_article(title, publication, api_key)
    # return record_string
    if record_string:
        article_details = extract_pubmed_details(record_string)
        return article_details
    else:
        return None

def add_pubmed_details(text_df, api_key):
    """
    Add the article metadata to a DataFrame containing article title and text.

    Parameters:
    - text_df (pd.DataFrame): DataFrame containing article title and text.
    - api_key (str): NCBI API key

    Returns:
    DataFrame with added PubMed details for each article.
    """
    article_details_list = []
    for index in text_df.index:
        article = text_df.loc[index, 'title']
        text = str(text_df.loc[index, 'body'])
        publication = text_df.loc[index, 'publication']
        article_details = pubmed_details_by_title(article, publication, api_key)
        if article_details:
            article_details['text'] = text
            article_details_list.append(article_details)
        else:
            article_details_list.append({
                'pubmed_title': article,
                'abstract': '',
                'journal': publication,
                'authors': '',
                'year': '',
                'month': '',
                'pub_volume': '',
                'pub_issue': '',
                'start_page': '',
                'end_page': '',
                'doi': '',
                'text': text,
                'section': section
            })
    article_details_df = pd.DataFrame(article_details_list)
    return pd.concat([text_df.reset_index(drop=True), article_details_df], axis=1)

def compare_columns(df, col1='title', col2='pubmed_title'):
    """
    Compare two columns in a DataFrame. Drop the second column if the two columns are identical.
    Otherwise, return the dataframe with new column with the comparison results, 
    where `True` indicates a mismatch.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the two columns to be compared.
    - col1 (str): Name of the first column to be compared.
    - col2 (str): Name of the second column to be compared.

    Returns:
    DataFrame with added column containing the comparison results.
    """
    # Remove punctuation and special characters
    remove_punct = lambda text: re.sub(f'[{string.punctuation}]', '', text)
    col1 = df[col1].apply(remove_punct)
    col2 = df[col2].apply(remove_punct)

    # Convert to lowercase and remove white spaces
    clean_text = lambda text: text.lower().strip()
    col1 = col1.apply(clean_text)
    col2 = col2.apply(clean_text)

    # Perform the comparison
    comparison = col1 != col2
    if sum(comparison) == 0:
        df = df.drop(columns=['pubmed_title'])
    else:
        df['flag_title'] = comparison
        flagged_indices = df[df['flag_title'] == True].index
        for index in flagged_indices:
            print(f'Flagged: ')
            print(f'\tArticle title: {df.loc[index, "title"]}')
            print(f'\tPubMed title: {df.loc[index, "pubmed_title"]}')
            print()
    
    return df

def create_sources_table(text_df, col1='title', col2='pubmed_title'):
    references_df = add_pubmed_details(text_df, api_key)

    references_df = compare_columns(references_df, col1=col1, col2=col2)
    return references_df


import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from db_session import *
from sqlalchemy.orm import declarative_base
from sqlalchemy import text
from sqlalchemy import Column, ForeignKey, Integer, String, Text, TIMESTAMP, Numeric, Boolean
from sqlalchemy.dialects.postgresql import UUID
import uuid
import pandas as pd
# from sqlalchemy.dialects.postgresql import insert
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

## This script calls upon functions and magic functions in db_session.py

Base = declarative_base()

class GPT_queue(Base):
    __tablename__ = 'gpt_queue'
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    body = mapped_column(Text)
    section = mapped_column(String(100))
    sent_to_sources = mapped_column(Boolean)
    publication = mapped_column(String(100))

class Sources(Base):
    __tablename__ = 'sources'
    id = mapped_column(Integer, primary_key=True)
    title = mapped_column(String(255))
    text = mapped_column(Text)
    abstract = mapped_column(Text)
    publication = mapped_column(String(100))
    authors = mapped_column(String(300))
    year = mapped_column(Integer)
    month = mapped_column(String(10))
    pub_volume = mapped_column(String(10))
    pub_issue = mapped_column(String(10))
    start_page = mapped_column(String(10))
    end_page = mapped_column(String(10))
    doi = mapped_column(String(50))
    section = mapped_column(String(100))
    summaries = relationship('Summaries', back_populates='sources')

class Prompts(Base):
    __tablename__ = 'prompts'
    id = mapped_column(Integer, primary_key=True)
    full_template = mapped_column(Text)
    system_role = mapped_column(String(300))
    prep_steps = mapped_column(Text)
    task = mapped_column(Text)
    edit_steps = mapped_column(Text)
    simplify_steps = mapped_column(Text)
    audience = mapped_column(String(200))
    format_steps = mapped_column(Text)

    summaries = relationship('Summaries', back_populates='prompts')
    
class Summaries(Base):
    __tablename__ = 'summaries'
    id = mapped_column(Integer, primary_key=True)
    timestamp = mapped_column(TIMESTAMP(timezone=True))
    original_summary = mapped_column(Text)
    rating_original_content = mapped_column(Integer) 
    simple_summary = mapped_column(Text)
    rating_simple_content = mapped_column(Integer) 
    original_headline = mapped_column(String(255))
    prompt_id = mapped_column(Integer, ForeignKey('prompts.id'), autoincrement=False)
    reference_id = mapped_column(Integer, ForeignKey('sources.id'), autoincrement=False)
    choice = mapped_column(Integer)
    model = mapped_column(String(70))
    temperature = mapped_column(Numeric)

    prompts = relationship('Prompts', back_populates='summaries')
    sources = relationship('Sources', back_populates='summaries')

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df

@remote_sql_session
def get_table(session, query='SELECT *', table='publications', limit=None, order_by='id', order='ASC'):
    """
    Return a database table as a pandas dataframe.
    """
    query_statement = f'{query} from {table}'
    if order_by:
        query_statement += f' ORDER BY {order_by} {order}'
    if limit:
        query_statement += f' LIMIT {limit}'
    print(f'Query: {query_statement}')
    q = session.execute(text(query_statement))
    df = pd.DataFrame(q.fetchall())
    return df


def bulk_append(input_df, table='summaries'):
    """
    Add articles to the `sources` table in the database from a dataframe containing article text and metadata.
    
    Parameters:
    - references_df: pandas dataframe containing article text and metadata.

    Returns: None
    """
    @remote_sql_session
    def insert_rows(session):
        try:
            print(f'Adding {len(input_df)} rows to the database...')
            def insert_row(row):
                if table == 'sources':
                    with session.no_autoflush:
                        existing_record = session.query(Sources).filter_by(
                            title=row['title'],
                            doi=row['doi'],
                            section=row['section']
                        ).first()
                        if not existing_record:
                            data = Sources(
                                title=row['title'],
                                text=row['text'],
                                abstract=row['abstract'],
                                publication=row['journal'],
                                authors=row['authors'],
                                year=row['year'],
                                month=row['month'],
                                pub_volume=row['pub_volume'],
                                pub_issue=row['pub_issue'],
                                start_page=row['start_page'],
                                end_page=row['end_page'],
                                doi=row['doi'],
                                section=row['section'] 
                            )
                            session.add(data)
                            print(f'\t{row["title"]}')
                        else:
                            print(f'\t** Already exists in the database: {row["title"]}.')
                elif table == 'gpt_queue':
                    data = GPT_queue(
                        title=row['title'],
                        body=row['body'],
                        section=row['section'],
                        sent_to_sources=row['sent_to_sources'],
                        publication=row['publication']
                    )
                    session.add(data)
                    print(f'\t{row["title"]}')
                elif table == 'summaries':
                    prompt = session.query(Prompts).filter_by(
                        full_template=row['full_summarize_task'],
                        system_role=row['system_role'],
                    ).first()
                    if prompt:
                        prompt_id = prompt.id
                    else:
                        prompt = Prompts(
                            full_template=row['full_summarize_task'],
                            prep_steps=row['prep_step'],
                            task=row['summarize_task'],
                            edit_steps=row['edit_task'],
                            audience=row['simplify_audience'],
                            simplify_steps=row['simplify_task'],
                            format_steps=row['format_task'],
                            system_role=row['system_role']
                        )
                        session.add(prompt)
                        session.flush()
                        prompt_id = prompt.id

                    summary = Summaries(
                        timestamp=row['timestamp'],
                        original_summary=row['summary'],
                        simple_summary=row['simple_summary'],
                        original_headline=row['headline'],
                        prompt_id=prompt_id,
                        reference_id=row['reference_id'],
                        choice=row['choice'],
                        model=row['model'],
                        temperature=row['temperature']
                    )
                    session.add(summary)
                    print(f'\tReference #{row["reference_id"]}: {row["headline"]}')
                elif table == 'feed':
                    source = session.query(Feed).filter_by(
                        title=row['title'],
                        journal=row['journal'],
                        doi=row['doi']
                    ).first()
                    if source:
                        print(f'\tAlready exists in the database: {row["title"]}.')

            input_df.apply(insert_row, axis=1)

            session.commit()
            print("New records added successfully (if applicable)!")
        except Exception as e:
            session.rollback()
            print(f"Error adding data to the database: {str(e)}")
        finally:
            session.close()

    return insert_rows()

from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from orm_summarize import *
from article_processing import *
# from sources import *
api_key = os.getenv('api_ncbi') # Pubmed API key

#########
#########
# Prep: Set parameters
folder_path = '../text/2023-05-03 5'
section = 'Sleep disturbance and obesity risk'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 2.2
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit, order='DESC') # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py
    # return references_df_dict[iteration_id]

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]



if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=local, article_limit=article_limit)


Query: SELECT * from gpt_queue ORDER BY id DESC LIMIT 1
Search term: (Sleep and obesity [ti]) AND (Curr Opin Clin Nutr Metab Care [ta])
Number of abstract sections: 3
hello


In [2]:
 qna_dict 

,id,title,body,sent_to_sources,section,publication,abstract,journal,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text
0,87,Sleep and obesity,Sleep and Obesity\r\n\r\nObstructive sleep apn...,None,Sleep disturbance and obesity risk,Curr Opin Clin Nutr Metab Care,PURPOSE OF REVIEW: This review summarizes the ...,Current opinion in clinical nutrition and meta...,"Guglielmo Beccuti, Silvana Pannain",2011,,14,4,402,412,10.1097/MCO.0b013e3283479109,Sleep and Obesity\r\n\r\nObstructive sleep apn...


In [3]:
qna_dict['journal'].values

array(['Current opinion in clinical nutrition and metabolic care'],
      dtype=object)

# *End of Page*